<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250725_OpenAI_test_%EC%B5%9C%EC%9D%80%ED%98%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os

base_path = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트"
for root, dirs, files in os.walk(base_path):
    print("📂", root)


📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/레퍼런스
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/재무제표 RAW 데이터 차트 추출
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/재무제표 RAW 데이터 차트 추출/매출 차트
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/250721_핵심피처(핵심컬럼) 후보 데이터 추출_호성, 은혜
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/250721_핵심피처(핵심컬럼) 후보 데이터 추출_호성, 은혜/사업보고서_23,24,25
📂 /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호

In [3]:
# ✅ 0. 필수 패키지
!pip install -q openai PyMuPDF


# ✅ 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 임포트
import os, re, unicodedata
import fitz  # PyMuPDF
import torch
from sentence_transformers import SentenceTransformer, util
import openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 64.3 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# ✅ 0. 필수 설치
!pip install --quiet openai PyMuPDF

# ✅ 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 임포트
import os, re, unicodedata
import fitz
import openai
import torch
from sentence_transformers import SentenceTransformer, util

# ✅ 3. OpenAI 최신 방식 (openai>=1.0.0)
client = openai.OpenAI(api_key="sk-...")  # ← 여기에 OpenAI API 키 입력

# ✅ 4. 사용자 입력
company = input("요약할 기업명을 입력하세요: ").strip()
company = unicodedata.normalize("NFC", company)

# ✅ 5. PDF 폴더 경로: 직접 지정
search_folder = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"
if not os.path.exists(search_folder):
    raise FileNotFoundError("❌ PDF 폴더 경로가 존재하지 않습니다.")

# ✅ 6. PDF 파일 검색
files = os.listdir(search_folder)
matched = [f for f in files if company in unicodedata.normalize("NFC", f) and f.lower().endswith(".pdf")]
if not matched:
    raise FileNotFoundError(f"❌ '{company}' 관련 PDF 파일을 찾을 수 없습니다.")
pdf_path = os.path.join(search_folder, matched[0])
print("✅ 찾은 PDF:", pdf_path)

# ✅ 7. PDF → 텍스트 추출
with fitz.open(pdf_path) as doc:
    raw_text = "".join([page.get_text() for page in doc])

# ✅ 8. 전처리
text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\s+', ' ', text)
sentences = re.split(r'(?<=[.?!])\s+', text)

def is_valid(s):
    s = s.strip()
    return len(s) > 10 and sum(c.isdigit() for c in s)/len(s) < 0.2 and len(re.findall(r'[가-힣a-zA-Z]', s)) >= 5

filtered = [s for s in sentences if is_valid(s)]
print(f"📚 유효 문장 수: {len(filtered)}")

# ✅ 9. 요약 주제
topics = {
    "📊 재무 요약": "최근 3개년 손익계산서를 기준으로 매출액, 영업이익, 당기순이익 등을 요약하세요.",
    "🛡️ 리스크 및 부채": "부채, 유동성 리스크, 지급보증, 담보 제공 등과 관련된 내용을 요약하세요.",
    "🚀 사업 전략": "신규 사업, 투자 계획, 성장 전략 등을 요약하세요.",
    "🔬 기술 및 R&D": "연구개발 현황, 기술 파이프라인, 기술 제휴 등을 요약하세요.",
    "🌐 글로벌 진출": "해외 시장 전략, 수출 확대, 해외 법인 관련 내용을 요약하세요.",
}

# ✅ 10. GPT 요약 함수
def gpt_summarize_section(text, topic_instruction):
    prompt = f"""다음은 한국 기업의 사업보고서 일부입니다.

요약 주제:
{topic_instruction}

아래 텍스트를 기반으로 한 문단(3~5문장)으로 요약해 주세요.

<본문>
{text[:2000]}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ 요약 실패: {e}"

# ✅ 11. 실행
text_sample = " ".join(filtered[:300])
results = {sec: gpt_summarize_section(text_sample, inst) for sec, inst in topics.items()}

# ✅ 12. 결과 출력 및 저장
output_txt = ""
for sec, summ in results.items():
    output_txt += f"\n\n## {sec}\n{summ}"

output_path = pdf_path.replace(".pdf", "_GPT요약.txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(output_txt.strip())

print("✅ GPT 요약 완료! 저장 위치:", output_path)
print(output_txt)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
요약할 기업명을 입력하세요: 두산
✅ 찾은 PDF: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/두산밥캣 사업보고서 (2024.12).pdf
📚 유효 문장 수: 2296
✅ GPT 요약 완료! 저장 위치: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/두산밥캣 사업보고서 (2024.12)_GPT요약.txt


## 📊 재무 요약
❌ 요약 실패: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-.... You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## 🛡️ 리스크 및 부채
❌ 요약 실패: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-.... You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}



## OpenAI 자연어처리 성공 코드

In [6]:
# ✅ 0. 필수 설치
!pip install --quiet openai PyMuPDF

# ✅ 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 임포트
import os, re, unicodedata
import fitz
import openai
import torch
from sentence_transformers import SentenceTransformer, util

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# ✅ 3. OpenAI 최신 방식 (openai>=1.0.0)
client = openai.OpenAI(api_key="키를 입력하세요")  # ← 본인의 키 입력

# ✅ 4. 사용자 입력
company = input("요약할 기업명을 입력하세요: ").strip()
company = unicodedata.normalize("NFC", company)

# ✅ 5. PDF 폴더 직접 지정
search_folder = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"
assert os.path.exists(search_folder), "❌ 지정한 PDF 폴더 경로가 존재하지 않습니다."

# ✅ 6. PDF 파일 탐색
files = os.listdir(search_folder)
matched = [f for f in files if company in unicodedata.normalize("NFC", f) and f.lower().endswith(".pdf")]
if not matched:
    raise FileNotFoundError(f"❌ '{company}' 관련 PDF 파일을 찾을 수 없습니다.")
pdf_path = os.path.join(search_folder, matched[0])
print("✅ 찾은 PDF:", pdf_path)

# ✅ 7. PDF → 텍스트 추출
with fitz.open(pdf_path) as doc:
    raw_text = "".join([page.get_text() for page in doc])

# ✅ 8. 텍스트 전처리 및 문장 필터링
text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\s+', ' ', text)
sentences = re.split(r'(?<=[.?!])\s+', text)

def is_valid(s):
    s = s.strip()
    return len(s) > 10 and sum(c.isdigit() for c in s)/len(s) < 0.2 and len(re.findall(r'[가-힣a-zA-Z]', s)) >= 5

filtered = [s for s in sentences if is_valid(s)]
print(f"📚 유효 문장 수: {len(filtered)}")

# ✅ 9. 요약 주제 정의
topics = {
    "📊 재무 요약": "최근 3개년 손익계산서를 기준으로 매출액, 영업이익, 당기순이익 등을 요약하세요.",
    "🛡️ 리스크 및 부채": "부채, 유동성 리스크, 지급보증, 담보 제공 등과 관련된 내용을 요약하세요.",
    "🚀 사업 전략": "신규 사업, 투자 계획, 성장 전략 등을 요약하세요.",
    "🔬 기술 및 R&D": "연구개발 현황, 기술 파이프라인, 기술 제휴 등을 요약하세요.",
    "🌐 글로벌 진출": "해외 시장 전략, 수출 확대, 해외 법인 관련 내용을 요약하세요.",
}

# ✅ 10. GPT 요약 함수
def gpt_summarize_section(text, topic_instruction):
    prompt = f"""다음은 한국 기업의 사업보고서 일부입니다.

요약 주제:
{topic_instruction}

아래 텍스트를 기반으로 한 문단(3~5문장)으로 요약해 주세요.

<본문>
{text[:2000]}"""  # 토큰 초과 방지

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ 요약 실패: {e}"

# ✅ 11. 요약 실행
text_sample = " ".join(filtered[:300])
results = {}
for sec, inst in topics.items():
    results[sec] = gpt_summarize_section(text_sample, inst)

# ✅ 12. 결과 출력 및 저장
output_txt = ""
for sec, summ in results.items():
    output_txt += f"\n\n## {sec}\n{summ}"

output_path = pdf_path.replace(".pdf", "_GPT요약.txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(output_txt.strip())

print("✅ GPT 요약 완료! 저장 위치:", output_path)
print(output_txt)


요약할 기업명을 입력하세요: 카카오
✅ 찾은 PDF: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/카카오 [기재정정]사업보고서 (2024.12).pdf
📚 유효 문장 수: 3499
✅ GPT 요약 완료! 저장 위치: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/카카오 [기재정정]사업보고서 (2024.12)_GPT요약.txt


## 📊 재무 요약
최근 3개년 손익계산서를 기준으로 한국 기업의 재무 성과를 요약하면 다음과 같습니다. 매출액은 지속적으로 증가하여 3년 전보다 약 15% 상승하였으며, 이는 주로 주요 제품 및 서비스의 수요 증가에 기인합니다. 영업이익 또한 매출 증가와 비용 절감 노력의 결과로 꾸준히 상승하여 영업이익률이 개선되었습니다. 당기순이익은 영업이익 증가와 함께 금융 비용 절감 등의 요인으로 인해 전반적으로 긍정적인 추세를 보이며, 주주들에게 안정적인 배당을 제공할 수 있는 기반을 마련하였습니다.

## 🛡️ 리스크 및 부채
이 회사의 사업보고서에 따르면, 부채 및 유동성 리스크와 관련하여 회사는 현재 부채 비율을 안정적으로 관리하고 있으며, 단기적인 유동성 문제를 해결하기 위해 다양한 금융 전략을 채택하고 있습니다. 지급보증과 관련해서는 주요 거래처와의 신뢰를 바탕으로 필요한 경우 지급보증을 제공하고 있으며, 이는 회사의 신용도를 유지하는 데 기여하고 있습니다. 또한, 담보 제공과 관련하여 회사는 필요한 경우 자산을 담보로 제공하여 금융기관으로부터 자금을 조달하고 있으며, 이는 재무 건전성을 유지하는 데 중요한 역할을 하고 있습니다.

## 🚀 사업 전략
이 한국 기업은 신규 사업으로 친환경 에